In [9]:
import pandas as pd

In [28]:
df = pd.read_csv('train_bodies.csv')
df.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [24]:
stop_words = set(stopwords.words('english'))

In [6]:
import collections
#a = collections.Counter(filtered_sent)

In [7]:
complete_df = df.merge(df_head,on='Body ID')

In [7]:
import re

In [11]:
from nltk.stem.porter import PorterStemmer

In [141]:
def text_processing(text):
    ps = PorterStemmer()
    head_list = []
    for h in text:
        h = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',h)
        h = ' '.join([ps.stem(i) for i in h.split(' ')])
        head_list.append(h.lower())
    head_list = [h.split(' ') for h in head_list]
    return head_list

In [129]:
text = ['(CNN) -- A meteorite: crashed down in? Managua, Nicaragua,@ late" Saturday’ night, cau$sing a loud explosion and leaving a crater 39 feet (12 meters) across, government officials said, according to The Associated Press.']
res = text_processing(text=text)
print(res)

(CNN) -- A meteorite: crashed down in? Managua, Nicaragua,@ late" Saturday’ night, cau$sing a loud explosion and leaving a crater 39 feet (12 meters) across, government officials said, according to The Associated Press.
[['cnn', '', 'a', 'meteorit', 'crash', 'down', 'in', 'managua', 'nicaragua', 'late', 'saturday', 'night', 'caus', 'a', 'loud', 'explos', 'and', 'leav', 'a', 'crater', '39', 'feet', '12', 'meter', 'across', 'govern', 'offici', 'said', 'accord', 'to', 'the', 'associ', 'press.']]


In [26]:
l = ["haven't","hadn't","not","don't","aren't","less","none","couldn't","doesn't","hasn't","isn't","mightn't","mustn't","needn't","shan't","shouldn't","wasn't","weren't","won't","wouldn't",""]
for w in l:
    try:
        stop_words.remove(w)
    except:
        pass

In [20]:
def stopword_removal(ls):
    filtered_list = []
    for hl in ls:
        temp = []
        for w in hl:
            if w not in stop_words:
                temp.append(w)
        filtered_list.append(temp)
    return filtered_list

In [178]:
covid_df = pd.read_csv('NLP_dataset_COVID_Economic_Corona.csv',encoding='latin-1')
covid_df

,Headline,articleBody,Stance
0,"Coronavirus Live Updates: India Records 1,112 ...","India's COVID-19 caseload climbed to 4,46,46,8...",agree
1,India vaccine maker destroys 100 million doses...,Indian vaccine maker Serum Institute of India ...,agree
2,New COVID-19 vaccination round to be launched ...,A new round of COVID-19 vaccinations for peopl...,agree
3,Book a coronavirus vaccination appointment in ...,"Starting on 24 October 2022, a vaccination app...",agree
4,Influenza vaccinations and new COVID-19 vaccin...,Influenza vaccinations and a new round of COVI...,agree
...,...,...,...
65,600 Sikh Pilgrims Not Allowed To Visit Pak Ove...,Washington: The top US health body has issued ...,unrelated
66,PSL Match Postponed After Islamabad United Spi...,Washington: The top US health body has issued ...,unrelated
67,Pakistan To Purchase 1.2 Million Covid Vaccine...,New Delhi: The Union Home Ministry Thursday de...,agree
68,Pakistan To Purchase 1.2 Million Covid Vaccine...,Two Hundred Sikh pilgrims from among 800 who w...,unrelated


In [187]:
a = []
b = []
c = []
count = 0
ps = PorterStemmer()
for _id in range(0,covid_df.shape[0]):
    #b_id = df_test_headline.iloc[_id]['Body ID']
    head_line = covid_df.iloc[_id]['Headline']
    #print(head_line)
    art_body = covid_df.iloc[_id]['articleBody']
    #head_line = list(df_test_headline[df_test_headline['Body ID']==_id]['Headline'])
    head_line = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',head_line)
    head_line = ' '.join([ps.stem(i) for i in head_line.split(' ')])
    art_body = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',art_body)
    art_body = ' '.join([ps.stem(i) for i in art_body.split(' ')])
    #preprossed_art_body = stopword_removal(ls=text_processing(art_body))
    #print(preprossed_art_body)
    #preprossed_head_line = stopword_removal(ls=text_processing(head_line))
    #print(preprossed_head_line)
    article_counter = collections.Counter(art_body)
    neg_body = []
    doc1 = nlp(art_body)
    for e in doc1.ents:
        neg_body.append(tuple([e.text,str(e._.negex)]))
    #for phl in preprossed_head_line:
    #h = ' '.join(phl)
    neg_head_line = []
    doc = nlp(head_line)
    for e in doc.ents:
        neg_head_line.append(tuple([e.text,str(e._.negex)]))
    count = 0
    head_line_counter = collections.Counter(head_line)
    try:
        for k in head_line_counter.keys():
            count = count+article_counter[k]
    except:
        count = count + 0
    if count==0:
        a.append(head_line)
        b.append(art_body)
        c.append('unrelated')
    else:
        if count>=len(head_line) and count>=5:
            a.append(head_line)
            b.append(art_body)
            c.append('agree')
        elif count<len(head_line) and count >= 1 and count < 5:
            a.append(head_line)
            b.append(art_body)
            c.append('discuss')
        else:
            for index, tup in enumerate(neg_headline_list):
                for i,j in enumerate(neg_body_list):
                    if index == i:
                        if tup != j:
                            a.append(head_line)
                            b.append(art_body)
                            c.append('disagree')

In [188]:
df = pd.DataFrame()
df['Headline'] = a
df['articleBody'] = b
df['Stance'] = c
df.head()

,Headline,articleBody,Stance
0,coronaviru live updat india record 1112 covid ...,india covid19 caseload climb to 44646880 with ...,agree
1,india vaccin maker destroy 100 million dose of...,indian vaccin maker serum institut of india si...,agree
2,new covid19 vaccin round to be launch for risk...,a new round of covid19 vaccin for peopl belong...,agree
3,book a coronaviru vaccin appoint in the maisa ...,start on 24 octob 2022 a vaccin appoint for al...,agree
4,influenza vaccin and new covid19 vaccin round ...,influenza vaccin and a new round of covid19 va...,agree


In [190]:
from sklearn.metrics import accuracy_score
accuracy_score(covid_df['Stance'],df['Stance'])

0.6

In [199]:
economy_df = pd.read_csv('NLP_dataset_COVID_Economic_Economic.csv',encoding='latin-1')
economy_df

,Headline,articleBody,Stance
0,Elon Musk says a global recession could last u...,Tesla founder and CEO Elon Musk thinks the glo...,agree
1,Hereâs how venture capital is helping to lif...,Even though Latinos are the second-largest eth...,discuss
2,"U.S. GDP accelerated at 2.6% pace in Q3, bette...",The U.S. economy posted its first period of po...,agree
3,"Tatas, Lockheed Martin in talks to make Sikors...",The Tatas are in talks with Lockheed Martin fo...,discuss
4,U.S. GDP bounces back but economic outlook rem...,The Commerce Department analysis found U.S. gr...,agree
...,...,...,...
91,Why didnât government apply for Debt Service...,We may never get official reasons for the Akuf...,discussion
92,Unite for Universal Hand Hygiene: A policy and...,Hand washing is the single most effective way ...,discussion
93,Winifred hMensa: The pure water price increase...,Let me preface this piece by saying that I am ...,discussion
94,"Samsonâs Take: No empty slogans on Sunday, M...","In the news, former NPP MP and anticorruption ...",discussion


In [201]:
a = []
b = []
c = []
count = 0
ps = PorterStemmer()
for _id in range(0,economy_df.shape[0]):
    #b_id = df_test_headline.iloc[_id]['Body ID']
    head_line = economy_df.iloc[_id]['Headline']
    #print(head_line)
    art_body = economy_df.iloc[_id]['articleBody']
    #head_line = list(df_test_headline[df_test_headline['Body ID']==_id]['Headline'])
    head_line = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',head_line)
    head_line = ' '.join([ps.stem(i) for i in head_line.split(' ')])
    art_body = re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',art_body)
    art_body = ' '.join([ps.stem(i) for i in art_body.split(' ')])
    #preprossed_art_body = stopword_removal(ls=text_processing(art_body))
    #print(preprossed_art_body)
    #preprossed_head_line = stopword_removal(ls=text_processing(head_line))
    #print(preprossed_head_line)
    article_counter = collections.Counter(art_body)
    neg_body = []
    doc1 = nlp(art_body)
    for e in doc1.ents:
        neg_body.append(tuple([e.text,str(e._.negex)]))
    #for phl in preprossed_head_line:
    #h = ' '.join(phl)
    neg_head_line = []
    doc = nlp(head_line)
    for e in doc.ents:
        neg_head_line.append(tuple([e.text,str(e._.negex)]))
    count = 0
    head_line_counter = collections.Counter(head_line)
    try:
        for k in head_line_counter.keys():
            count = count+article_counter[k]
    except:
        count = count + 0
    if count==0:
        a.append(head_line)
        b.append(art_body)
        c.append('unrelated')
    else:
        if count>=len(head_line) or count>=5:
            a.append(head_line)
            b.append(art_body)
            c.append('agree')
        elif count<len(head_line) or count >= 1 and count < 5:
            a.append(head_line)
            b.append(art_body)
            c.append('discuss')
        else:
            for index, tup in enumerate(neg_headline_list):
                for i,j in enumerate(neg_body_list):
                    if index == i:
                        if tup != j:
                            a.append(head_line)
                            b.append(art_body)
                            c.append('disagree')

In [195]:
df = pd.DataFrame()
df['Headline'] = a
df['articleBody'] = b
df['Stance'] = c
df.head()

,Headline,articleBody,Stance
0,elon musk say a global recess could last until...,tesla founder and ceo elon musk think the glob...,agree
1,hereâ how ventur capit is help to lift the n...,even though latino are the secondlargest ethni...,agree
2,u.s. gdp acceler at 2.6% pace in q3 better tha...,the u.s. economi post it first period of posit...,agree
3,tata lockhe martin in talk to make sikorski s7...,the tata are in talk with lockhe martin for ma...,agree
4,u.s. gdp bounc back but econom outlook remain ...,the commerc depart analysi found u.s. gross do...,agree


In [200]:
df['Stance'].unique()

array(['agree'], dtype=object)

In [210]:
y_actual = economy_df.iloc[0:92,:]['Stance']

In [211]:
from sklearn.metrics import accuracy_score
accuracy_score(y_actual,df['Stance'])

0.31521739130434784

In [212]:
df_test_body = pd.read_csv('test_bodies.csv')
df_test_headline = pd.read_csv('test_stances_unlabeled.csv')
data = df_test_body.merge(df_test_headline,on='Body ID')

In [159]:
data

,Body ID,articleBody,Headline,Stance
0,1,"India's COVID-19 caseload climbed to 4,46,46,8...","Coronavirus Live Updates: India Records 1,112 ...",agree
1,2,Indian vaccine maker Serum Institute of India ...,India vaccine maker destroys 100 million doses...,agree
2,3,A new round of COVID-19 vaccinations for peopl...,New COVID-19 vaccination round to be launched ...,agree
3,5,Influenza vaccinations and a new round of COVI...,Influenza vaccinations and new COVID-19 vaccin...,agree
4,6,The results of the third external study on the...,The city's crisis management model for dealing...,agree
...,...,...,...,...
83,93,The Story India has a relatively open economy....,"CCI fines Google ?2,274 crores in a week",agree
84,94,The Story India has a relatively open economy....,"CCI gives Google ?2,274 crores in a week",disagree
85,95,The Story There are about 20 quadrillion ants ...,The Extraordinary Lives of Ants,discuss
86,96,Key Highlights of the Economic Survey 2018-19 ...,"Economic Survey 2019 Analysis: 7% GDP growth, ...",agree


In [213]:
a = []
b = []
c = []
count = 0
for _id in range(0,100):
    try:
        #b_id = df_test_headline.iloc[_id]['Body ID']
        head_line = list(df_test_headline[df_test_headline['Body ID']==_id]['Headline'])
        #print(head_line)
        art_body = list(df_test_body[df_test_body['Body ID']==_id]['articleBody'])
        #head_line = list(df_test_headline[df_test_headline['Body ID']==_id]['Headline'])
        preprossed_art_body = stopword_removal(ls=text_processing(art_body))[0]
        #print(preprossed_art_body)
        preprossed_head_line = stopword_removal(ls=text_processing(head_line))
        print(preprossed_head_line)
        article_counter = collections.Counter(preprossed_art_body)
        neg_body = []
        doc1 = nlp(' '.join(preprossed_art_body))
        for e in doc1.ents:
            neg_body.append(tuple([e.text,str(e._.negex)]))
        for phl in preprossed_head_line:
            h = ' '.join(phl)
            neg_head_line = []
            doc = nlp(' '.join(phl))
            for e in doc.ents:
                neg_head_line.append(tuple([e.text,str(e._.negex)]))
            count = 0
            head_line_counter = collections.Counter(phl)
            try:
                for k in head_line_counter.keys():
                    count = count+article_counter[k]
            except:
                count = count + 0
            if count==0:
                a.append(head_line)
                b.append(art_body)
                c.append('unrelated')
            else:
                if count>=len(phl) and count>=5:
                    a.append(head_line)
                    b.append(art_body)
                    c.append('agree')
                elif count<len(phl) and count >= 1 and count < 5:
                    a.append(head_line)
                    b.append(art_body)
                    c.append('discuss')
                else:
                    for index, tup in enumerate(neg_headline_list):
                        for i,j in enumerate(neg_body_list):
                            if index == i:
                                if tup != j:
                                    a.append(head_line)
                                    b.append(art_body)
                                    c.append('disagree')
    except:
        pass

[['appl', 'instal', 'safe', 'instor', 'protect', 'gold', 'watch', 'edit'], ['elsisi', 'deni', 'claim', 'hell', 'give', 'sinai', 'land', 'palestinian'], ['appl', 'keep', 'gold', 'watch', 'edit', 'special', 'instor', 'safe'], ['appl', 'store', 'keep', 'gold', '“edition”', 'appl', 'watch', 'custom', 'safe'], ['south', 'korean', 'woman', 'hair', 'eaten', 'robot', 'vacuum', 'cleaner', 'slept'], ['18karat', 'gold', 'appl', 'watch', 'edit', 'lock', 'safe', 'appl', 'store'], ['appl', 'watch', 'price', 'releas', 'date', 'new', 'special', 'safe', 'appl', 'store', 'gold', 'model', 'april', 'launch', 'confirm'], ['appl', 'plan', 'phase', 'beat', 'music', 'brand', 'report', 'say'], ['appl', 'store', 'customdesign', 'safe', 'store', 'gold', 'appl', 'watch'], ['polish', 'teenag', 'wake', 'dure', 'brain', 'surgeri', 'talk', 'doctor'], ['cnn', 'doctor', 'took', 'midsurgeri', 'selfi', 'unconsci', 'joan', 'river'], ['gold', 'appl', 'watch', 'store', 'special', 'safe', 'appl', 'store'], ['teen', 'wake', '

[['islam', 'state', 'leader', 'abu', 'bakr', 'albaghdadi', 'wound', 'airstrik', 'claim'], ['charl', 'manson', 'fiance', 'allegedli', 'want', 'marri', 'hi', 'corps'], ['tran', 'teen', 'die', 'suicid', 'leav', 'tumblr', 'note', 'way'], ['lisa', 'bonet', 'not', 'send', '“cryptic', 'tweet”', 'bill', 'cosbi', '—', 'twitter', 'account', 'fake!'], ['transgend', 'teen', 'struck', 'kill', 'ohio', 'interst', 'appar', 'suicid'], ['transgend', 'teen', 'commit', 'suicid', 'left', 'heartbreak', 'note', 'say', 'would', 'never', 'happi'], ['ohio', 'tran', 'teen', 'commit', 'suicid', 'leav', 'heartbreak', 'note', 'declar', '“there', 'way', 'out”'], ['charl', 'manson', 'fiance', 'use', 'hi', '...', 'corps'], ['found!', 'miss', 'afghan', 'soldier', 'spot', 'tri', 'enter', 'canada'], ['fox', 'affili', 'fall', 'colleg', 'humor', 'offic', 'note', 'war', 'prank'], ['report', 'nhl', 'expans', 'la', 'vega', 'done', 'deal'], ['4inch', 'iphon', '6', 'report', 'develop'], ['transgend', 'teen', 'leelah', 'alcorn',

[['microsoft', 'near', 'deal', 'buy', 'minecraft', 'maker', 'mojang', '', 'wsj'], ['minecraft', 'expect', 'acquir', 'microsoft'], ['appl', 'scrap', 'advanc', 'appl', 'watch', 'health', 'monitor', 'featur', 'due', 'reliabl', 'issu'], ['microsoft', 'reportedli', 'buy', 'minecraft', 'develop', 'mojang', '2', 'billion'], ['argentina', 'presid', 'adopt', 'jewish', 'boy', 'save', 'life', 'werewolf'], ['ye', 'bought', 'microsoft'], ['microsoft', 'buy', 'minecraft', 'creator', 'mojang', '2.5b'], ['report', 'microsoft', 'tri', 'buy', 'mojang', 'creator', 'minecraft'], ['microsoft', 'near', 'deal', 'buy', 'minecraft', 'maker', 'mojang'], ['report', 'microsoft', 'may', 'buy', 'minecraft', 'studio', 'mojang'], ['microsoft', 'seriou', 'discuss', 'purchas', 'minecraft', 'dev', 'mojang'], ['microsoft', 'said', 'buy', 'maker', 'minecraft', '2', 'billion'], ['[googl', 'translate]', 'site', 'famili', 'flyer', 'maryam', 'mansouri', 'emphas', 'fabric', 'statement', 'repudi']]
[['thi', 'crabzilla', 'spot',

[['white', 'hous', 'chief', 'staff', 'deni', 'mcdonough', 'threat', 'made', 'foley', 'sotloff', 'famili', 'possibl', 'ransom'], ['wh', 'didnt', 'threaten', 'foley', 'sotloff', 'famili'], ['isi', 'reportedli', 'behead', 'poster', 'child', 'kurdish', 'femal', 'fighter'], ['white', 'hous', 'threaten', 'jame', 'foley', 'famili', 'prosecut'], ['jame', 'foley', 'famili', 'obama', 'offici', 'vow', 'crimin', 'charg', 'paid', 'son', 'ransom'], ['isi', 'advanc', 'within', '10', 'mile', 'baghdad'], ['appl', 'reportedli', 'plan', 'shut', 'beat', 'music', 'subscript', 'stream', 'servic', '[u]'], ['jame', 'foley', 'famili', 'claim', 'govern', 'threaten', 'ransom', 'payment'], ['report', 'woman', 'claim', 'third', 'breast', 'ad', 'hope', 'becom', 'tv', 'star'], ['thi', 'man', 'kept', 'harass', 'women', 'streets.', 'one', 'day…', 'mom!'], ['us', 'threaten', 'prosecut', 'famili', '2', 'murder', 'american', 'paid', 'ransom', 'isi'], ['girl', 'destroy', 'boyfriend', 'peni', 'acid', 'attack', 'video', 'se

[['hewlettpackard', 'break', '2', 'compani'], ['report', 'marin', 'order', 'state', 'dept', 'surrend', 'weapon', 'us', 'embassi', 'evac', 'yemen'], ['yemen', 'houthi', 'rebel', 'seiz', 'u.s.', 'marin', 'weapon', 'airport'], ['miss', 'afghan', 'soldier', 'taken', 'custodi', 'canadian', 'border'], ['hewlettpackard', 'split', 'pc', 'printer', 'busi'], ['hewlettpackard', 'plan', 'split', 'two', 'compani', 'report'], ['hewlett', 'packard', 'plan', 'split', 'two'], ['u.s.', 'offici', 'threaten', 'jame', 'foley', 'parent', 'prosecut', 'ransom'], ['pete', 'hegseth', 'marin', 'exit', 'yemen', 'forc', 'surrend', 'digniti', 'weapon'], ['hp', 'split', 'two', 'busi', '', 'report'], ['hp', 'better', 'not', 'togeth', '—', 'compani', 'split', 'enterpris', 'pc/printer', 'busi'], ['hewlettpackard', 'reportedli', 'split', 'two', 'compani'], ['scorpion', 'sting', 'woman', 'osu', 'plane'], ['hp', 'reportedli', 'plan', 'split', 'pc', 'printer', 'busi', 'enterpris', 'unit'], ['fox', 'new', 'float', 'bs', 'st

[['get', 'detail', 'amaz', 'new', 'camera', 'appl', 'plan', 'next', 'iphon']]
[['transgend', 'teenag', '17', 'leav', 'heartbreak', 'suicid', 'note', 'blame', 'christian', 'parent', 'befor', 'walk', 'front', 'tractor', 'trailer', 'highway'], ['transgend', 'teen', 'leav', 'note', 'tumblr', 'blame', 'christian', 'parent', 'suicid'], ['transgend', 'teen', 'leelah', 'alcorn', 'commit', 'suicid', 'walk', 'front', 'semi', 'truck', 'blame', 'christian', 'parent', 'viral', 'suicid', 'note'], ['felt', 'threaten', 'open', 'umbrella', 'rais', 'hand'], ['transgend', 'teen', 'leelah', 'alcorn', 'blame', 'christian', 'parent', 'tumblr', 'suicid', 'note'], ['islam', 'state', 'leader', 'abu', 'bakr', 'albaghdadi', 'wound', 'airstrik', 'claim'], ['argentina', 'presid', 'didnt', 'adopt', 'jewish', 'child', 'avoid', 'becom', 'werewolf;'], ['tran', 'teen', 'die', 'suicid', 'leav', 'tumblr', 'note', 'way'], ['stop', 'share', 'photo', 'fanci', 'intern', 'school', 'lunch'], ['transgend', 'teen', 'commit', 'su

[['ibm', 'sack', '111800', 'employe', 'record', 'mass', 'layoff'], ['ibm', 'job', 'cuts;', 'd.r.', 'horton', 'soar', 'beat;', 'at&t', 'expand', 'mexico'], ['report', 'isi', 'islamist', 'gate', 'baghdad'], ['joan', 'river', 'doctor', 'snap', 'selfi', 'dure', 'throat', 'procedur', 'report'], ['miss', 'nuke', 'colorado', 'spark', 'intens', 'russian', 'fear'], ['ibm', 'corp', 'expect', 'cut', '110000', 'job', 'massiv', 'overhaul', 'report', 'say'], ['disgusting!', 'joan', 'river', 'doc', 'gwen', 'korovin', 'sick', 'selfi', 'expos', '—', 'last', 'photo', 'comic', 'icon', 'wa', 'anesthesia', 'center', 'medic', 'investig'], ['ibm', 'plan', 'massiv', 'layoff', '110000', 'worker', 'report'], ['ibm', 'set', 'cut', '110000', 'job', 'blogger', 'claim'], ['ibm', 'deni', 'layoff', 'rumor'], ['gay', 'teacher', 'receiv', 'touch', 'letter', 'student', '9', 'come'], ['poster', 'girl', 'fight', 'isi', 'behead', 'terror', 'group'], ['isi', 'terror', 'chief', 'believ', 'dead', 'iraqi', 'militari', 'confirm

[['valentin', 'day', '2015', 'may', 'appl', 'watch', 'releas', 'target'], ['appl', 'watch', 'like', 'launch', 'valentin', 'day;', 'plu', 'first', 'appl', 'watch', 'accessori', 'showcas', 'ce', '2015'], ['joan', 'river', 'doctor', 'snap', 'selfi', 'dure', 'throat', 'surgeri', 'report'], ['hermaphrodit', 'cat', 'get', 'gender', 'assign', 'surgeri'], ['ghost', 'ship'], ['rumor', 'appl', 'watch', 'product', 'gear', 'valentin', 'day', 'releas', 'high', 'price']]
[['hand', 'dont', 'shoot', 'hong', 'kong', 'protest'], ['gigafactori', 'locat', 'tesla', 'motor', 'confirm', 'nevada', 'announc', 'thursday'], ['stolen', 'member'], ['happen', 'hong', 'kong', 'stun', 'imag', 'occupyinspir', 'fergusonting', 'protest'], ['felt', 'threaten', 'open', 'umbrella', 'rais', 'hand'], ['hong', 'kong', 'protest', 'use', 'hand', 'dont', 'shoot', 'gestur', 'ferguson'], ['“hand', 'dont', 'shoot”', 'come', 'hong', 'kong', 'prodemocraci', 'movement'], ['hong', 'kong', 'protest', 'mimic', 'ferguson', 'hand', 'dont',

In [215]:
df = pd.DataFrame()
df['Headline'] = a
df['articleBody'] = b
df['Stance'] = c
df

,Headline,articleBody,Stance
0,[Apple installing safes in-store to protect go...,[Al-Sisi has denied Israeli reports stating th...,unrelated
1,[Apple installing safes in-store to protect go...,[Al-Sisi has denied Israeli reports stating th...,discuss
2,[Apple installing safes in-store to protect go...,[Al-Sisi has denied Israeli reports stating th...,unrelated
3,[Apple installing safes in-store to protect go...,[Al-Sisi has denied Israeli reports stating th...,unrelated
4,[Apple installing safes in-store to protect go...,[Al-Sisi has denied Israeli reports stating th...,unrelated
...,...,...,...
893,"[‘Hands Up, Don’t Shoot’ in Hong Kong Protests...","[Wen Apple first released the iPhone and iPad,...",unrelated
894,"[‘Hands Up, Don’t Shoot’ in Hong Kong Protests...","[Wen Apple first released the iPhone and iPad,...",unrelated
895,"[‘Hands Up, Don’t Shoot’ in Hong Kong Protests...","[Wen Apple first released the iPhone and iPad,...",unrelated
896,"[‘Hands Up, Don’t Shoot’ in Hong Kong Protests...","[Wen Apple first released the iPhone and iPad,...",discuss


In [219]:
pred = pd.read_csv('test_pred.csv')
pred.head()

,Headline,Body ID,stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated


In [230]:
ls = []
for _id in range(0,100):
    try:
        ls.append(list(pred[pred['Body ID']==_id]['stance']))
    except:
        pass

In [236]:
y_actual = pd.DataFrame()
ls1 = []
for i in ls:
    for j in i:
        ls1.append(j)
y_actual['Stance'] = ls1

In [240]:
y_actual = y_actual.iloc[0:898,:]['Stance']

In [242]:
from sklearn.metrics import accuracy_score
accuracy_score(y_actual,df['Stance'])

0.621380846325167

In [164]:
predict_df = pd.DataFrame()
predict_df['Headline'] = list(d.keys())
predict_df['Stance'] = list(d.values())

In [166]:
predict_df['Stance'].unique()

array(['agree', 'discuss', 'unrelated'], dtype=object)

In [154]:
y_actual = pd.read_csv('predictions_test.csv')
y_actual = y_actual.head(832)

In [169]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_actual['Stance'],predict_df['Stance'])
acc

ValueError: Found input variables with inconsistent numbers of samples: [832, 62]

In [157]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_actual['Stance'],predict_df['Stance'])
cm

array([[  2,   0,   9,  25],
       [  0,   0,   0,   1],
       [ 25,   0,  65,  91],
       [ 77,   0, 207, 330]], dtype=int64)

End of the code

In [119]:
import re
t = '(CNN) -- A meteorite: crashed down in? Managua, Nicaragua,@ late" Saturday’ night, cau$sing a loud explosion and leaving a crater 39 feet (12 meters) across, government officials said, according to The Associated Press.'
print(re.sub(r"\n|\'|-|,|\)|\(|\:|\?|@|‘|’|\*|\"|\$",'',t))

CNN  A meteorite crashed down in Managua Nicaragua late Saturday night causing a loud explosion and leaving a crater 39 feet 12 meters across government officials said according to The Associated Press.


In [2]:
pip install tensorflow

  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached termcolor-2.0.1-py3-none-any.whl (5.4 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (15

In [6]:
import tensorflow.compat.v1 as tf

In [7]:
tf.disable_v2_behavior()
x = tf.placeholder(shape=[None, 2], dtype=tf.float32)

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
x

<tf.Tensor 'Placeholder:0' shape=(?, 2) dtype=float32>

In [72]:
negation_list = ['not','no','none','less']
neg_stop_words = []
print(list(stop_words))
for sw in list(stop_words):
    if sw not in negation_list:
        neg_stop_words.append(sw)
print(neg_stop_words)

['re', 'a', 'and', 'into', 'no', 'only', 'aren', 'hadn', "couldn't", "weren't", 'than', 'mustn', 'him', 'or', 'herself', "won't", 'after', 'my', 'o', 'each', 'has', 'shan', 'hers', 'that', "hadn't", "it's", 'then', 'didn', 'me', 'am', 'will', 'further', 'under', 'y', 's', 'once', 'before', 'again', 'her', 'himself', 'below', "that'll", 'against', 'same', 'isn', 'he', 'm', "you'll", 'while', 'were', 'more', 'do', 'shouldn', 'have', 'yours', "shan't", "didn't", 'here', 'which', 'any', 'myself', 'ours', 'from', 'won', 'ourselves', 'this', 'about', 'you', "haven't", 'his', "you're", 'such', 'needn', 'i', 'out', 'some', 'as', 'did', 'between', 'theirs', 'who', 'in', 'had', 'weren', 'them', 'of', 'yourself', 'our', 'most', 'their', 't', 'is', 've', 'over', "should've", 'now', "she's", 'other', 'themselves', 'can', 'very', 'itself', 'yourselves', 'she', 'why', "wouldn't", 'its', 'doesn', 'just', 'we', 'those', 'are', 'doing', 'wouldn', 'until', 'having', 'on', 'an', 'all', "isn't", 'mightn', 

In [6]:
!pip install spacy

  Attempting uninstall: smart-open
    Found existing installation: smart-open 5.1.0
    Uninstalling smart-open-5.1.0:
      Successfully uninstalled smart-open-5.1.0


In [8]:
!pip install negspacy

  Created wheel for negspacy: filename=negspacy-1.0.3-py3-none-any.whl size=10434 sha256=3ffa193a3fb3b5fd88f4b5007242c591ac3a4296643622d27bb9374fb7b0915e
  Stored in directory: c:\users\fuzel\appdata\local\pip\cache\wheels\ec\4b\5b\ae823d4b232c1342724ec93edab9f3dca6403679d93e0daa25
Successfully built negspacy


In [10]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-10-29 15:27:36.850694: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-29 15:27:36.850789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-29 15:27:41.907516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-29 15:27:41.907554: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-29 15:27:41.913394: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: Fuzel
2022-10-29 15:27:41.913626: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: Fuzel


In [14]:
import spacy
from negspacy.negation import Negex

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("negex", config={"ent_types":["PERSON","ORG"]})

doc = nlp("")
for e in doc.ents:
    print(e.text, e._.negex)

In [23]:
a = {'a':True,'b':False,'c':True}
b = {'a':True,'e':True,'c':False}
for a_k in a.keys():
    if a_k in b.keys() and b[a_k]==a[a_k]:
        print(a_k,'Agree')
    elif a_k in b.keys() and b[a_k]!=a[a_k]:
        print(a_k,'Disagree')
    else:
        print(a_k,'unrelated')

a Agree
b unrelated
c Disagree


In [25]:
pred_csv = pd.read_csv('predictions_test.csv')

In [26]:
pred_csv.head()

,Stance
0,unrelated
1,unrelated
2,unrelated
3,unrelated
4,unrelated


In [27]:
test_csv = pd.read_csv('test_stances_unlabeled.csv')

In [28]:
test_csv['stance'] = pred_csv

In [29]:
test_csv

,Headline,Body ID,stance
0,Ferguson riots: Pregnant woman loses eye after...,2008,unrelated
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,1550,unrelated
2,A Russian Guy Says His Justin Bieber Ringtone ...,2,unrelated
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",1793,unrelated
4,Argentina's President Adopts Boy to End Werewo...,37,unrelated
...,...,...,...
25408,The success of the Affordable Care Act is a hu...,2582,agree
25409,The success of the Affordable Care Act is a hu...,2583,discuss
25410,The success of the Affordable Care Act is a hu...,2584,unrelated
25411,The success of the Affordable Care Act is a hu...,2585,discuss


In [32]:
test_csv.to_csv('test_pred.csv',index=False)